In [12]:
# import sys
# print(sys.version)
# !/home/rongxiang/miniconda3/envs/env_gcn/bin/pip install scikit-learn

In [11]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [13]:
%%time

df_traffic = pd.read_csv('output/austin_2022_GP_10min_interval_delaydifference_with_features_forML2.csv')
df_traffic = df_traffic.rename(columns={"miles": "segment_length"})

df_traffic['minutes_since_midnight'] = df_traffic['hour_min'].apply(lambda x: int(x[:2]) * 60 + int(x[3:]))

print(df_traffic.date.unique())
print('unique road segments (samples):',df_traffic.tmc_code.unique().shape[0])
print('total observations: ',df_traffic.shape[0])
df_traffic.head(2)

['2022-10-21' '2022-10-22' '2022-10-23']
unique road segments (samples): 3221
total observations:  1388251
CPU times: user 5.64 s, sys: 986 ms, total: 6.63 s
Wall time: 7.44 s


,tmc_code,hour_min,delay_baseline,delay_focus,delay_difference,date,intersection,start_latitude,start_longitude,end_latitude,...,distance_to_Shuttle_Expo_Center,segment_bearing,osm_id,road_type,oneway,lanes,maxspeed,lanes_mean,minutes_since_midnight,no_devices
0,112+04760,00:00,-1.779261,-2.222261,-0.443000,2022-10-21,US-183/EXIT 239-240,30.32874,-97.70568,30.33272,...,10.337892,19.798294,"[122981397, 1012363973]66847798019339769835",secondary_link,True,"['1', '2']",55 mph,1.5,0,0.0
1,112+04760,00:10,-1.309413,-2.434401,-1.124988,2022-10-21,US-183/EXIT 239-240,30.32874,-97.70568,30.33272,...,10.337892,19.798294,"[122981397, 1012363973]66847798019339769835",secondary_link,True,"['1', '2']",55 mph,1.5,10,0.0


In [16]:
# because the event usually begins in the morning after 10, people arrive typically around that time, so we want to only focus on after 930
df_traffic = df_traffic[df_traffic['minutes_since_midnight']>=540]
print(df_traffic.shape[0])
df_traffic = pd.get_dummies(df_traffic, columns=['date'], prefix='date')

# df_traffic_oneday = df_traffic[df_traffic['date']=='2022-10-22']
# df_traffic['distance_to_Uber_DelValle_HighSchool'] = df_traffic['distance_to_venue_centroid']
df_traffic.loc[df_traffic.road_type=="motorway_link", 'road_type'] = 'motorway'
df_traffic.loc[df_traffic.road_type=="primary_link", 'road_type'] = 'primary'
df_traffic.loc[df_traffic.road_type=="secondary_link", 'road_type'] = 'secondary'
df_traffic.loc[df_traffic.road_type=="trunk_link", 'road_type'] = 'trunk'
df_traffic.loc[df_traffic.road_type=="tertiary_link", 'road_type'] = 'tertiary'
df_traffic['road_type'].value_counts()

866449


road_type
secondary       334905
motorway        176464
tertiary        162745
primary         122126
residential      35508
trunk            24479
unclassified     10222
Name: count, dtype: int64

In [17]:
import re

def process_max_speed(value):
    if isinstance(value, str):
        # Extract all numeric values from the string
        numbers = list(map(int, re.findall(r'\d+', value)))
        if numbers:
            return max(numbers)  # Return the maximum value
    return None

# Apply the function to the max_speed column
df_traffic['max_speed'] = df_traffic['maxspeed'].apply(process_max_speed)

In [20]:
feature_columns = ['minutes_since_midnight',   'segment_length','segment_bearing', 'oneway', 'lanes_mean','road_type', 'max_speed',
                'date_2022-10-21', 'date_2022-10-22', 'date_2022-10-23',
                'airbnb_count',  'distance_to_venue_centroid', 'no_devices'
               ] 

# Prepare input features and labels for the model
X = df_traffic[feature_columns].copy() 
# 'distance_to_Shuttle_Waterloo_Park', 'distance_to_Shuttle_Barton_Creek_Square', 'distance_to_Shuttle_Expo_Center',
# 'start_latitude', 'start_longitude',

X['road_type'] = X['road_type'].astype('category')

y = df_traffic['delay_difference'].values

# Normalize features
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# Create graph structure

In [ ]:
# Create edges (source and target road segment IDs)
edges = df[['source_id', 'target_id']].values.T
edges = torch.tensor(edges, dtype=torch.long)

# Create node features (scaled road attributes)
node_features = torch.tensor(X_scaled, dtype=torch.float)

# Create target tensor
targets = torch.tensor(y, dtype=torch.float)